# 1. Introduction

Over the past three missions, we learned about decision trees, and looked at ways to reduce overfitting. The most powerful tool for reducing decision tree overfitting is called the random forest algorithm. In this mission, we'll learn how to construct and apply random forests.

We'll continue using the same 1994 census data set on U.S. income. It contains information on marital status, age, type of work, and more. The target column, high_income, records salaries less than or equal to 50k a year (0), and more than 50k a year (1).

You can download the data from the University of California, Irvine's website.

In [22]:
import pandas
income = pandas.read_csv('income2.csv')
# Convert a single column from text categories to numbers
col = pandas.Categorical.from_array(income["workclass"])
income["workclass"] = col.codes
print(income["workclass"].head(5))
for name in ["education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "high_income"]:
    col = pandas.Categorical.from_array(income[name])
    income[name] = col.codes

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8


C:\Users\mciniello\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
C:\Users\mciniello\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: Categorical.from_array is deprecated, use Categorical instead


In [23]:
import numpy
import math

# Set a random seed so the shuffle is the same every time
numpy.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income = income.reindex(numpy.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

# 2. Combining Model Predictions With Ensembles

A random forest is a kind of ensemble model. Ensembles combine the predictions of multiple models to create a more accurate final prediction. We'll make a simple ensemble to see how they work.

Let's create two decision trees with slightly different parameters:

- One with min_samples_leaf set to 2
- One with max_depth set to 5

Then, we'll check their accuracies separately. On the next screen, we'll combine their predictions and compare the combined accuracy with the individual accuracies of both trees.

**Instructions**

- Fit both clf and clf2 to the data.
    - Use train[columns] as the predictors, and train["high_income"] as the target.
- Make predictions on the test set predictors (test[columns]) using both clf and clf2.
- For both sets of predictions, compute the AUC between the predictions and the actual values (test["high_income"]) using the roc_auc_score function.
- Use the print() function to display the AUC values for both.

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])

clf2 = DecisionTreeClassifier(random_state=1, max_depth=5)
clf2.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

predictions = clf2.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.687896422606
0.675985390651


# 3. Combining Our Predictions

When we have multiple classifiers making predictions, we can treat each set of predictions as a column in a matrix. Here's an example where we have Decision Tree 1 (DT1), Decision Tree 2 (DT2), and Decision Tree 3 (DT3):

    DT1     DT2    DT3
    0       1      0
    1       1      1
    0       0      1
    1       0      0
    
Whenever we add more models to our ensemble, we just add more columns to the combined predictions. Ultimately, we don't want this matrix, though -- we want one prediction per row in the training data. To accomplish this, we'll need to create rules to convert each row of our matrix of predictions into a single number.

We want to create a Final Prediction vector that looks like this:

    DT1     DT2    DT3    Final Prediction
    0       1      0      0
    1       1      1      1
    0       0      1      0
    1       0      0      0
    
There are many ways to get from the output of multiple models to a final vector of predictions. One method is majority voting, in which each classifier gets a "vote," and the most commonly voted value for each row "wins." This only works if there are more than two classifiers (and ideally an odd number, so we don't have to write a rule to break ties). Majority voting is what we applied in the example above.

Because we only had two classifiers on the last screen, we'll have to use a different method to combine predictions. We'll take the mean of all of the items in a row. Right now, we're using the predict() method, which returns either 0 or 1. predict() returns something like this:

    0
    1
    0
    1
    
We can use the RandomForestClassifier.predict_proba() method instead, which will predict a probability from 0 to 1 that a given class is the right one for a row. Because 0 and 1 are our two classes, we'll get a matrix containing the number of rows in the income dataframe, and two columns. predict_proba() will return a result that looks like this:

    0     1
    .7    .3
    .2    .8
    .1    .9
    
Each row will correspond to a prediction. The first column is the probability that the prediction is a 0, and the second column is the probability that the prediction is a 1. Each row adds up to 1.

If we just take the second column, we get the average value that the classifier would predict for that row. If there's a .9 probability that the correct classification is 1, we can use the .9 as the value the classifier is predicting. This will give us a continuous output in a single vector, instead of just 0 or 1.

Then we can add together all of the vectors we get through this method, and divide the sum by the total number of vectors to get the mean prediction made across the entire ensemble for a particular row. Finally, we round off to get a 0 or 1 prediction for the row.

If we use the predict_proba() method on both classifiers from the last screen to generate probabilities, take the mean for each row, and then round the results, we'll get ensemble predictions.

# 4. Combining Our Predictions

- Add predictions and predictions2, then divide by 2 to get the mean.
- Use numpy.round() to round all of the resulting predictions.
- Print the resulting AUC score between the actual values and the predictions.

In [25]:
predictions = clf.predict_proba(test[columns])[:,1]
predictions2 = clf2.predict_proba(test[columns])[:,1]
combined = (predictions + predictions2) / 2
rounded = numpy.round(combined)

print(roc_auc_score(test["high_income"], rounded))

0.715084680404


# 5. Why Ensembling Works
As we can see from the previous screen, the combined predictions of the two trees had a higher AUC than that of either tree on its own:
<table style="width:50%">
  <tr>
    <th>Settings</th>
    <th>Test AUC</th> 
  </tr>
  <tr>
    <td>min_samples_leaf: 2</td>
    <td>0.688</td>
  </tr>
  <tr>
    <td>max_depth: 2</td>
    <td>0.676</td>
  </tr>
  <tr>
    <td>combined predictions</td>
    <td>0.715</td>
  </tr>
</table>

To intuitively understand why this makes sense, think about two people with the same level of talent. One learned programming in college, while the other learned it on her own (let's say using Dataquest!).

If we give both of them the same project, they'll approach it in slightly different ways, due to their different experiences. They may both produce code that achieves the same result, but one may run faster in certain areas. The other may have a better interface. Even though both of them have about the same level of talent, their solutions are stronger in different areas because they approached the problem differently.

If we combine the best parts of both of their projects, we'll end up with a stronger combined project.

Ensembling is exactly the same. The models are approaching the same problem in slightly different ways, and building different trees because we used different parameters for each one. Each tree makes different predictions in different areas. Even though both trees have about the same accuracy, when we combine them, the result is stronger because it leverages the strengths of both approaches.

<font color=red> The more "diverse" or dissimilar the models we use to construct an ensemble are, the stronger their combined predictions will be (assuming that all of the models have about the same accuracy). Ensembling a decision tree and a logistic regression model, for example, will result in stronger predictions than ensembling two decision trees with similar parameters. That's because those two models use very different approaches to arrive at their answers.</font>

On the other hand, if the models we ensemble are very similar in how they make predictions, ensembling will result in a negligible boost.

**Ensembling models with very different accuracies generally won't improve overall accuracy. Ensembling a model with a .75 AUC and a model with a .85 AUC on a test set will usually result in an AUC somewhere in between the two original values. There's a way around this that we'll discuss later on, which we call weighting.**

# 6. Introducing Variation With Bagging

A random forest is an ensemble of decision trees. If we don't make any modifications to the trees, each tree will be exactly the same, so we'll get no boost when we ensemble them. In order to make ensembling effective, we have to introduce variation into each individual decision tree model.

If we introduce variation, each tree will be be constructed slightly differently, and will therefore make different predictions. This variation is what puts the "random" in "random forest."

There are two main ways to introduce variation in a random forest -- bagging and random feature subsets. We'll dive into bagging first.

In a random forest, we don't train each tree on the entire data set. We train it on a random sample of the data, or a "bag," instead. We perform this sampling with replacement, which means that after we select a row from the data we're sampling, we put the row back in the data so it can be picked again. Some rows from the original data may appear in the "bag" multiple times.

Let's use bagging with the first tree we trained.

**Instructions:**

- predictions is a list of vectors corresponding to predictions on the test set.
- Use the technique we employed earlier to add all of the vectors together, and divide by 10 to get the mean prediction for each row.
- Use numpy.round() to round the resulting predictions.
- Finally, print the AUC score between the combined predictions and test["high_income"].

In [26]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample in every loop
    # That would make all of our trees the same
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])
combined = numpy.sum(predictions, axis=0) / 10
rounded = numpy.round(combined)

print(roc_auc_score(test["high_income"], rounded))

0.732996329747


# 7. Selecting Random Features
Using the bagging example from the previous screen, we gained some accuracy over a single decision tree. To be exact, we achieved an AUC score of around .733 with bagging, which is an improvement over the AUC score of .688 we got without bagging:

<table style="width:50%">
  <tr>
    <th>Settings</th>
    <th>Test AUC</th> 
  </tr>
  <tr>
    <td>min_samples_leaf: 2</td>
    <td>0.688</td>
  </tr>
  <tr>
    <td>max_depth: 2</td>
    <td>0.676</td>
  </tr>
  <tr>
    <td>combined predictions</td>
    <td>0.715</td>
  </tr>
  <tr>
    <td>min_samples_leaf: 2, with bagging</td>
    <td>0.732</td>
  </tr>
</table>

Let's go back to the decision tree algorithm we explored two missions ago to explain random feature subsets:

- First we pick the maximum number of features we want to evaluate each time we split the tree.
    - This has to be less than the total number of columns in the data.
- Every time we split, we pick a random sample of features from the data.
- Then we compute the information gain for each feature in our random sample, and pick the one with the highest information gain to split on.

To understand how splits work, let's look at information gain or entropy. Entropy is the measure of "disorder" in the data set. If a dataset has all the same labels, they'll have low entropy. If all the labels are different, they'll have high entropy. Splits that give us more information about the data, will ideally minimize entropy. In other words, the tree will ideally split the labels into distinct groups with as little mixture possible. This'll allow the splits to give our tree more predictive power.

We're repeating the same process to select the optimal split that minimizes entropy for a node. However, we'll only evaluate a constrained set of features that we select randomly. This introduces variation into the trees, and makes for more powerful ensembles.

Below is the ID3 algorithm that we developed earlier. We'll modify it to only consider a certain subset of the features.

**Instructions**

- Modify find_best_column to select a random sample from columns before computing information gain.
    - Look where it says Insert code here.
    - Each subset will have 2 items in it.
    - You can use numpy.random.choice() to select a random sample.
    - The first input is the list you're picking from, and the second is the number of items you want to pick.
- Be careful not to overwrite columns when you do the selection.
    - The random sample should go in a different variable, and you'll have to modify some of the surrounding code to match.
- Use the print() function to display tree


In [32]:
import numpy as np
import math
# first re-load the entropy function
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

In [65]:
# Create the data set that we used two missions ago
data = pandas.DataFrame([
    [0,4,20,0],
    [0,4,60,2],
    [0,5,40,1],
    [1,4,25,1],
    [1,5,35,2],
    [1,5,55,1]
    ])
data.columns = ["high_income", "employment", "age", "marital_status"]

# Set a random seed to make the results reproducible
numpy.random.seed(1)

# The dictionary to store our tree
tree = {}
nodes = []

# The function to find the column to split on
def find_best_column(data, target_name, columns):
    information_gains = []
    
    # Insert your code here
    
    for col in columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    # Find the name of the column with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain

# The function to construct an ID3 decision tree
def id3(data, target, columns, tree):
    unique_targets = pandas.unique(data[target])
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1 in unique_targets:
            tree["label"] = 1
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])


# Run the ID3 algorithm on our data set and print the resulting tree
id3(data, "high_income", ["employment", "age", "marital_status"], tree)

def find_best_column(data, target_name, columns):
    information_gains = []
    
    # Select two columns randomly
    cols = numpy.random.choice(columns, 2)
    
    for col in cols:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    highest_gain_index = information_gains.index(max(information_gains))
    
    # Get the highest gain by indexing "cols"
    highest_gain = cols[highest_gain_index]
    
    return highest_gain

id3(data, "high_income", ["employment", "age", "marital_status"], tree)
tree

{'column': 'age',
 'left': {'column': 'employment',
  'left': {'column': 'age',
   'left': {'label': 0, 'number': 15},
   'median': 22.5,
   'number': 14,
   'right': {'label': 1, 'number': 16}},
  'median': 4.0,
  'number': 13,
  'right': {'label': 1, 'number': 17}},
 'median': 37.5,
 'number': 12,
 'right': {'column': 'age',
  'left': {'column': 'age',
   'left': {'label': 0, 'number': 20},
   'median': 47.5,
   'number': 19,
   'right': {'label': 1, 'number': 21}},
  'median': 55.0,
  'number': 18,
  'right': {'label': 0, 'number': 22}}}

# 8. Radnome Subsets in scikit-learn

We can also repeat our random subset selection process in scikit-learn. We just set the splitter parameter on DecisionTreeClassifier to "random", and the max_features parameter to "auto". If we have N columns, this will pick a subset of features of size $\sqrt{N}$, compute the Gini coefficient for each (this is similar to information gain), and split the node on the best column in the subset.

This is essentially the same thing we did on the previous screen, but with far less typing.

**Instructions**

- Modify the instantiation of the DecisionTreeClassifier object.
    - Set splitter to "random", and max_features to "auto".
- Print the resulting AUC score.

In [68]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample every time
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2, splitter="random", max_features="auto")
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

combined = numpy.sum(predictions, axis=0) / 10
rounded = numpy.round(combined)

print(roc_auc_score(test["high_income"], rounded))

0.7345958638


# 9. Practice Putting it All Together

Using random subsets from the previous screen improved the accuracy over using bagging alone:

<table style="width:50%">
  <tr>
    <th>Settings</th>
    <th>Test AUC</th> 
  </tr>
  <tr>
    <td>min_samples_leaf: 2</td>
    <td>0.688</td>
  </tr>
  <tr>
    <td>max_depth: 2</td>
    <td>0.676</td>
  </tr>
  <tr>
    <td>combined predictions</td>
    <td>0.715</td>
  </tr>
  <tr>
    <td>min_samples_leaf: 2, with bagging</td>
    <td>0.732</td>
  </tr>
  <tr>
    <td>min_samples_leaf: 2, with bagging AND randome subsets</td>
    <td>0.735
  </td>
  </tr></table>

So far, we've demonstrated the two building blocks of random forests, bagging and random feature subsets. Luckily, we don't have to write code from scratch each time. Scikit-learn has a RandomForestClassifier class and a RandomForestRegressor class that enable us to train and test random forest models quickly.

When we instantiate a RandomForestClassifier, we pass in an n_estimators parameter that indicates how many trees to build. While adding more trees usually improves accuracy, it also increases the overall time the model takes to train.

RandomForestClassifier has a similar interface to DecisionTreeClassifier, and we can use the fit() and predict() methods to train and make predictions.

**Instructions**

- Fit clf to the training data and make predictions on the test data.
- Compute and print the AUC score between the test predictions and the actual values.

In [70]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=5, random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.734746139194


# 10. Tweaking Parameters to Increase Accuracy

Similar to decision trees, we can tweak some of the parameters for random forests, including:

- min_samples_leaf
- min_samples_split
- max_depth
- max_leaf_nodes

These parameters apply to the individual trees in the model, and change how they are constructed. There are also parameters specific to the random forest that alter its overall construction:

- n_estimators
- bootstrap - "Bootstrap aggregation" is another name for bagging; this parameter indicates whether to turn it on (Defaults to True)
- Refer to the documentation for a full list of parameters.

Tweaking parameters can increase the accuracy of the forest. The easiest tweak is to increase the number of estimators we use. This approach yields diminishing returns -- going from 10 trees to 100 will make a bigger difference than going from 100 to 500, which will make a bigger difference than going from 500 to 1000. The accuracy increase function is logarithmic, so increasing the number of trees beyond a certain number (usually 200) won't help much at all.

In [72]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=2)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.737940321312


# 11. Reducing Overfitting

We were able to improve the AUC from 0.735 to 0.738, but the model using 150 trees took much longer to train. While the extra training time is trivial on the data set we're working with right now, understanding this trade-off will help you when you're working with much larger data sets where the extra time could amount to hours or days!

One of the major advantages of random forests over single decision trees is that they tend to overfit less. Although each individual decision tree in a random forest varies widely, the average of their predictions is less sensitive to the input data than a single tree is. This is because while one tree can construct an incorrect and overfit model, the average of 100 or more trees will be more likely to hone in on the signal and ignore the noise. The signal will be the same across all of the trees, whereas each tree will hone in on the noise differently. This means that the average will discard the noise and keep the signal.

In the following code cell, you'll see that we've fit a single decision tree to the training set, and made predictions for both the training and testing sets. The AUC for the training set predictions is .819, while the AUC for the testing set is .714. The fact that the test AUC is much lower than the train AUC indicates that the model is overfitting.

Now let's train a similar random forest model and contrast it with what we just did.

**Instructions**

- Fit clf to the training set and use it to make predictions on the training set.
- Then, use it to make predictions on the testing set.
- Print both AUC scores.

In [73]:
clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=5)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.791704729514
0.749887434396


# 12. When to Use Randome Forests

As we can see in the code cell from the previous screen, overfitting decreased with a random forest, and accuracy went up overall.

While the random forest algorithm is incredibly powerful, it isn't applicable to all tasks. The main strengths of a random forest are:

- Very accurate predictions - Random forests achieve near state-of-the-art performance on many machine learning tasks. Along with neural networks and gradient-boosted trees, they're typically one of the top-performing algorithms.
- Resistance to overfitting - Due to their construction, random forests are fairly resistant to overfitting. We still need to set and tweak parameters like max_depth though.

The main weaknesses of using a random forest are:

- They're difficult to interpret - Because we've averaging the results of many trees, it can be hard to figure out why a random forest is making predictions the way it is.
- They take longer to create - Making two trees takes twice as long as making one, making three takes three times as long, and so on. Fortunately, we can exploit multicore processors to parallelize tree construction. **Scikit allows us to do this through the n_jobs parameter on RandomForestClassifier. We'll discuss parallelization in greater detail later on.**

Given these trade-offs, it makes sense to use random forests in situations where accuracy is of the utmost importance; being able to interpret or explain the decisions the model is making isn't key. In cases where time is of the essence or interpretability is important, a single decision tree may be a better choice.

Next up in this course is a guided project where you'll explore using random forests to predict bike rentals.